In [1]:
#!pip install selenium
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [2]:
import os
os.chdir(r'C:\Users\Navid\Documents\ds_salary_proj')

In [3]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
def get_jobs(num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized");
    options.add_argument("disable-infobars")
    options.add_argument("--disable-extensions")
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="/Users/Navid/Documents/ds_salary_proj/chromedriver", options=options)
#     driver.set_window_size(1200, 2400)

    url = 'https://www.glassdoor.ca/Job/us-data-scientist-jobs-SRCH_IL.0,2_IN1_KO3,17.htm?fromAge=1'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(10)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_css_selector("[alt='Close']").click()  #clicking to the X.
        except NoSuchElementException:
            pass
        
#         WebDriverWait(driver, 20).until(ec.visibility_of_element_located((By.XPATH, 
#            '//a[@class = "close"]')))

#         close =  driver.find_element_by_xpath("//a[@class = 'close']/*[name()='svg']")

#         actionChains = ActionChains(driver)
#         actionChains.double_click(close).perform()

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            driver.execute_script("arguments[0].click();", job_button)  #You might 
            time.sleep(2)
            
            try:
                driver.find_element_by_css_selector("[alt='Close']").click()  #clicking to the X.
            except NoSuchElementException:
                pass
            
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(4)

            try:
                salary_range = driver.find_element_by_xpath('.//div[@class="salary"]//span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_range = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."
                
#             try:
#                 driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="salary"]').click()  
                
#                 try:
#                     salary_estimate = driver.find_element_by_xpath('.//div[@class="strong"]').text
#                 except NoSuchElementException:
#                     salary_estimate = -1
                    
#             except NoSuchElementException:
#                 salary_estimate = -1

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Range" : salary_range,           
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [4]:
df = get_jobs(400, False)

Progress: 0/400
Progress: 1/400
Progress: 2/400
Progress: 3/400
Progress: 4/400
Progress: 5/400
Progress: 6/400
Progress: 7/400
Progress: 8/400
Progress: 9/400
Progress: 10/400
Progress: 11/400
Progress: 12/400
Progress: 13/400
Progress: 14/400
Progress: 15/400
Progress: 16/400
Progress: 17/400
Progress: 18/400
Progress: 19/400
Progress: 20/400
Progress: 21/400
Progress: 22/400
Progress: 23/400
Progress: 24/400
Progress: 25/400
Progress: 26/400
Progress: 27/400
Progress: 28/400
Progress: 29/400
Progress: 30/400
Progress: 31/400
Progress: 32/400
Progress: 33/400
Progress: 34/400
Progress: 35/400
Progress: 36/400
Progress: 37/400
Progress: 38/400
Progress: 39/400
Progress: 40/400
Progress: 41/400
Progress: 42/400
Progress: 43/400
Progress: 44/400
Progress: 45/400
Progress: 46/400
Progress: 47/400
Progress: 48/400
Progress: 49/400
Progress: 50/400
Progress: 51/400
Progress: 52/400
Progress: 53/400
Progress: 54/400
Progress: 55/400
Progress: 56/400
Progress: 57/400
Progress: 58/400
Progres

In [23]:
df.Headquarters.value_counts()

-1    190
Name: Headquarters, dtype: int64

In [5]:
df.to_csv('df_data_scientist7')

In [24]:
df.shape

(190, 14)

In [47]:
df[df['Salary Estimate'] == -1].shape

(80, 15)